In [ ]:
# Import all the libraries I need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')  # getting tired of the deprecation warnings

# Stats and ML stuff
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Set seed so results are consistent
np.random.seed(42)

# Plot settings - took me a while to figure out the right style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("All libraries loaded!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")


In [ ]:
def generate_synthetic_data(n_users=75000):
    """
    Creates fake user data that behaves like real gamified learning app users
    Had to make this realistic enough to show actual patterns
    """
    
    # Make user IDs - keeping it simple with zero padding
    user_ids = [f"USER_{str(i).zfill(6)}" for i in range(1, n_users + 1)]
    
    # Signup dates over past 6 months
    start_date = datetime.now() - timedelta(days=180)
    end_date = datetime.now() - timedelta(days=1)
    
    # More signups in recent months (exponential growth pattern)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    weights = np.exp(np.linspace(0, 2, len(date_range)))  # this took me forever to get right
    signup_dates = np.random.choice(date_range, size=n_users, p=weights/weights.sum())
    
    # A/B test groups - 50/50 split between morning and evening notifications
    notification_times = np.random.choice(['morning', 'evening'], size=n_users, p=[0.5, 0.5])
    
    # Now the tricky part - making engagement metrics that actually correlate
    
    # Lesson completion - evening users do better (this is our key finding)
    base_lessons = np.random.poisson(lam=8, size=n_users)
    # Give evening users a boost - tried different values, 2 seems realistic
    evening_boost = np.where(notification_times == 'evening', 
                           np.random.poisson(lam=3, size=n_users), 0)
    lessons_completed = np.maximum(0, base_lessons + evening_boost)
    
    # Streak achievement - should correlate with lessons
    streak_probability = np.clip(lessons_completed / 20, 0, 0.85)
    streak_milestone = np.random.binomial(1, streak_probability, size=n_users).astype(bool)
    
    # Daily sessions - people with streaks are more engaged
    base_sessions = np.random.gamma(2, 1.5, size=n_users)
    streak_bonus = np.where(streak_milestone, np.random.gamma(1, 0.8, size=n_users), 0)
    daily_sessions = np.round(np.maximum(0.1, base_sessions + streak_bonus), 1)
    
    # Churn - inversely related to engagement (obviously)
    engagement_score = (lessons_completed * 0.4 + 
                       streak_milestone.astype(int) * 3 + 
                       daily_sessions * 0.8)
    
    # Higher engagement = lower churn
    churn_probability = np.clip(0.6 - (engagement_score / 30), 0.05, 0.95)
    churn_status = np.random.binomial(1, churn_probability, size=n_users).astype(bool)
    
    # Put it all together
    df = pd.DataFrame({
        'user_id': user_ids,
        'signup_date': signup_dates,
        'lesson_completed': lessons_completed,
        'streak_milestone': streak_milestone,
        'daily_sessions': daily_sessions,
        'notification_time': notification_times,
        'churn_status': churn_status
    })
    
    return df

# Generate the data
print("Creating synthetic dataset with 75K users...")
df_raw = generate_synthetic_data(n_users=75000)

print(f"Done! Dataset shape: {df_raw.shape}")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\\nFirst few rows:")
print(df_raw.head())


In [ ]:
# Let me check what we actually got
print("=== QUICK DATA CHECK ===")
print(f"Total Users: {len(df_raw):,}")
print(f"Date Range: {df_raw['signup_date'].min().strftime('%Y-%m-%d')} to {df_raw['signup_date'].max().strftime('%Y-%m-%d')}")
print(f"Days Covered: {(df_raw['signup_date'].max() - df_raw['signup_date'].min()).days}")

print("\\n=== COLUMN INFO ===")
print(df_raw.info())

# Quick sanity check on the data
print("\\n=== BASIC STATS ===")
print(df_raw.describe())

print("\\n=== DESCRIPTIVE STATISTICS ===")
print(df_raw.describe())

print("\\n=== CATEGORICAL VARIABLES ===")
print(f"Notification Time Distribution:")
print(df_raw['notification_time'].value_counts())
print(f"\\nStreak Milestone Achievement:")
print(df_raw['streak_milestone'].value_counts())
print(f"\\nChurn Status:")
print(df_raw['churn_status'].value_counts())

print("\\n=== DATA QUALITY CHECKS ===")
print(f"Missing Values:")
print(df_raw.isnull().sum())
print(f"\\nDuplicate User IDs: {df_raw['user_id'].duplicated().sum()}")
print(f"Negative Values Check: {(df_raw.select_dtypes(include=[np.number]) < 0).sum().sum()}")

# Quick correlation check
print("\\n=== CORRELATION MATRIX ===")
correlation_matrix = df_raw.select_dtypes(include=[np.number]).corr()
print(correlation_matrix.round(3))


In [ ]:
# Quick exploration - let me see what's going on with notification groups
print("Notification time split:")
print(df_raw['notification_time'].value_counts())
print()

# Are evening users actually different?
print("Average lessons by notification time:")
print(df_raw.groupby('notification_time')['lesson_completed'].mean())
print()

# Check churn rates by group
print("Churn rates:")
print(df_raw.groupby('notification_time')['churn_status'].mean())

# This looks promising - evening users do seem to complete more lessons!


In [ ]:
def preprocess_and_engineer_features(df):
    """
    Clean up the data and create some useful features
    This function does a lot - probably could break it down but it works
    """
    
    # Make a copy so I don't mess up the original
    df_processed = df.copy()
    
    print("=== CLEANING AND FEATURE ENGINEERING ===")
    
    # 1. Check for missing data
    print(f"1. Missing Data Check:")
    missing_data = df_processed.isnull().sum()
    print(f"   Missing values: {missing_data.sum()}")
    
    # Handle missing values if we have any
    if missing_data.sum() > 0:
        print("   Filling missing values...")
        # Use median for lesson_completed - seems more robust than mean
        if 'lesson_completed' in df_processed.columns:
            df_processed['lesson_completed'].fillna(
                df_processed['lesson_completed'].median(), inplace=True)
        
        # Use mean for daily_sessions 
        if 'daily_sessions' in df_processed.columns:
            df_processed['daily_sessions'].fillna(
                df_processed['daily_sessions'].mean(), inplace=True)
    
    # 2. Optimize data types to save memory
    print("2. Data Type Optimization:")
    print(f"   Original memory: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Make sure dates are actually dates
    df_processed['signup_date'] = pd.to_datetime(df_processed['signup_date'])
    
    # Lessons completed doesn't need to be int64 - int16 is plenty
    df_processed['lesson_completed'] = df_processed['lesson_completed'].astype('int16')
    
    # Boolean columns
    df_processed['streak_milestone'] = df_processed['streak_milestone'].astype('bool')
    df_processed['churn_status'] = df_processed['churn_status'].astype('bool')
    
    # Daily sessions as float32 (don't need double precision)
    df_processed['daily_sessions'] = df_processed['daily_sessions'].round(1).astype('float32')
    
    # Notification time as category - saves space
    df_processed['notification_time'] = df_processed['notification_time'].astype('category')
    
    print(f"   Optimized memory: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Saved: {((df.memory_usage(deep=True).sum() - df_processed.memory_usage(deep=True).sum()) / df.memory_usage(deep=True).sum() * 100):.1f}%")
    
    # 3. Feature Engineering - this is where it gets interesting
    print("3. Creating New Features:")
    current_date = datetime.now()
    
    # How long have users been with us?
    df_processed['days_since_signup'] = (current_date - df_processed['signup_date']).dt.days
    print(f"   - days_since_signup: {df_processed['days_since_signup'].min()}-{df_processed['days_since_signup'].max()} days")
    
    # Cumulative lessons (just copying for now but could be more complex)
    df_processed['cumulative_lessons'] = df_processed['lesson_completed']
    print(f"   - cumulative_lessons: max {df_processed['cumulative_lessons'].max()}")
    
    # Engagement score - weighted combination of different activities
    # Took me a while to get these weights right
    df_processed['engagement_score'] = (
        df_processed['lesson_completed'] * 0.4 +
        df_processed['streak_milestone'].astype(int) * 3.0 +
        df_processed['daily_sessions'] * 0.8 +
        (df_processed['days_since_signup'] / 30) * 0.2  # longer tenure = slight bonus
    )
    print(f"   - engagement_score: {df_processed['engagement_score'].min():.2f} to {df_processed['engagement_score'].max():.2f}")
    
    # User segments based on engagement - these thresholds are somewhat arbitrary
    def get_user_segment(row):
        score = row['engagement_score']
        if score >= 15:
            return 'High_Engagement'
        elif score >= 8:
            return 'Medium_Engagement' 
        elif score >= 3:
            return 'Low_Engagement'
        else:
            return 'At_Risk'
    
    df_processed['user_segment'] = df_processed.apply(get_user_segment, axis=1)
    df_processed['user_segment'] = df_processed['user_segment'].astype('category')
    print(f"   - user_segment: {list(df_processed['user_segment'].unique())}")
    
    # Churn risk score (flip of engagement score)
    max_engagement = df_processed['engagement_score'].max()
    df_processed['churn_risk_score'] = 1 - (df_processed['engagement_score'] / max_engagement)
    df_processed['churn_risk_score'] = df_processed['churn_risk_score'].clip(0, 1)
    print(f"   - churn_risk_score: {df_processed['churn_risk_score'].min():.3f} to {df_processed['churn_risk_score'].max():.3f}")
    
    # Time-based features for cohort analysis
    df_processed['signup_month'] = df_processed['signup_date'].dt.strftime('%Y-%m')
    df_processed['signup_month'] = df_processed['signup_month'].astype('category')
    print(f"   - signup_month: {df_processed['signup_month'].nunique()} months")
    
    df_processed['signup_week'] = df_processed['signup_date'].dt.to_period('W').astype(str)
    df_processed['signup_week'] = df_processed['signup_week'].astype('category')
    print(f"   - signup_week: {df_processed['signup_week'].nunique()} weeks")
    
    # 4. Quick validation
    print("4. Data Validation:")
    
    # Spot check - users with lots of lessons should probably have streaks
    high_lessons_no_streak = ((df_processed['lesson_completed'] > 15) & 
                             (~df_processed['streak_milestone'])).sum()
    print(f"   - Users with >15 lessons but no streak: {high_lessons_no_streak} (seems low, good)")
    
    # Check engagement distribution
    print(f"   - Engagement quartiles:")
    print(f"     25%: {df_processed['engagement_score'].quantile(0.25):.2f}")
    print(f"     50%: {df_processed['engagement_score'].quantile(0.5):.2f}")
    print(f"     75%: {df_processed['engagement_score'].quantile(0.75):.2f}")
    
    print(f"   - No missing values: {df_processed.isnull().sum().sum() == 0}")
    print(f"   - Final shape: {df_processed.shape}")
    
    return df_processed

# Apply preprocessing pipeline
print("Starting preprocessing and feature engineering...")
df_processed = preprocess_and_engineer_features(df_raw)

print("\\n=== PREPROCESSING COMPLETE ===")
print("New columns added:")
new_columns = set(df_processed.columns) - set(df_raw.columns)
for col in sorted(new_columns):
    print(f"  - {col}")


In [ ]:
# Let me just check what the user segments look like
print("User segment breakdown:")
print(df.groupby('user_segment').size())
print()

# And see how they differ by notification time
print("Segments by notification time:")
segment_table = pd.crosstab(df['notification_time'], df['user_segment'])
print(segment_table)
print()

# Calculate percentages - this might be interesting for the business case
print("Segment percentages by notification time:")
print(pd.crosstab(df['notification_time'], df['user_segment'], normalize='index').round(3) * 100)


In [ ]:
def analyze_conversion_funnel(df):
    """
    Analyze user conversion funnel across key engagement stages
    
    Parameters:
    df (pd.DataFrame): Processed user data
    
    Returns:
    dict: Funnel analysis results
    """
    
    print("=== FUNNEL ANALYSIS ===")
    
    # Define funnel stages
    total_signups = len(df)
    
    # Stage 1: Signup → First Lesson (at least 1 lesson completed)
    first_lesson_users = df[df['lesson_completed'] >= 1]
    first_lesson_count = len(first_lesson_users)
    
    # Stage 2: First Lesson → Active Learner (5+ lessons)
    active_learners = df[df['lesson_completed'] >= 5]
    active_learner_count = len(active_learners)
    
    # Stage 3: Active Learner → Streak Achiever
    streak_achievers = df[(df['lesson_completed'] >= 5) & (df['streak_milestone'] == True)]
    streak_achiever_count = len(streak_achievers)
    
    # Calculate conversion rates
    signup_to_first_lesson = (first_lesson_count / total_signups) * 100
    first_lesson_to_active = (active_learner_count / first_lesson_count) * 100 if first_lesson_count > 0 else 0
    active_to_streak = (streak_achiever_count / active_learner_count) * 100 if active_learner_count > 0 else 0
    
    # Overall conversion rate (signup to streak achiever)
    overall_conversion = (streak_achiever_count / total_signups) * 100
    
    # Print results
    print(f"Total Signups: {total_signups:,}")
    print(f"\\nFUNNEL STAGES:")
    print(f"1. Signup → First Lesson:")
    print(f"   Users: {first_lesson_count:,} ({signup_to_first_lesson:.1f}%)")
    print(f"   Drop-off: {total_signups - first_lesson_count:,} users")
    
    print(f"\\n2. First Lesson → Active Learner (5+ lessons):")
    print(f"   Users: {active_learner_count:,} ({first_lesson_to_active:.1f}%)")
    print(f"   Drop-off: {first_lesson_count - active_learner_count:,} users")
    
    print(f"\\n3. Active Learner → Streak Achiever:")
    print(f"   Users: {streak_achiever_count:,} ({active_to_streak:.1f}%)")
    print(f"   Drop-off: {active_learner_count - streak_achiever_count:,} users")
    
    print(f"\\nOVERALL CONVERSION RATE:")
    print(f"Signup → Streak Achiever: {overall_conversion:.1f}%")
    
    # Funnel data for visualization
    funnel_data = {
        'stage': ['Signups', 'First Lesson', 'Active Learner', 'Streak Achiever'],
        'users': [total_signups, first_lesson_count, active_learner_count, streak_achiever_count],
        'conversion_rate': [100.0, signup_to_first_lesson, first_lesson_to_active, active_to_streak]
    }
    
    return funnel_data

# Analyze conversion funnel
funnel_results = analyze_conversion_funnel(df_processed)

# Create funnel visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Funnel chart
stages = funnel_results['stage']
users = funnel_results['users']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Bar chart showing absolute numbers
bars = ax1.bar(stages, users, color=colors, alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_title('User Conversion Funnel - Absolute Numbers', fontsize=14, fontweight='bold')
ax1.set_ylabel('Number of Users')
ax1.set_xlabel('Funnel Stage')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height):,}',
             ha='center', va='bottom', fontweight='bold')

# Format y-axis with comma separator
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))

# Conversion rate line chart
conversion_rates = [100, funnel_results['conversion_rate'][1], 
                   funnel_results['conversion_rate'][2], funnel_results['conversion_rate'][3]]

ax2.plot(stages, conversion_rates, marker='o', linewidth=3, markersize=8, color='red')
ax2.set_title('Stage-by-Stage Conversion Rates', fontsize=14, fontweight='bold')
ax2.set_ylabel('Conversion Rate (%)')
ax2.set_xlabel('Funnel Stage')
ax2.grid(True, alpha=0.3)

# Add percentage labels
for i, rate in enumerate(conversion_rates):
    ax2.text(i, rate + 1, f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')

ax2.set_ylim(0, 110)

plt.tight_layout()
plt.show()

# Calculate drop-off analysis
print("\\n=== DROP-OFF ANALYSIS ===")
total_dropoffs = funnel_results['users'][0] - funnel_results['users'][-1]
print(f"Total users lost through funnel: {total_dropoffs:,}")
print(f"Overall retention rate: {(funnel_results['users'][-1] / funnel_results['users'][0] * 100):.1f}%")

stage_dropoffs = []
for i in range(len(funnel_results['users'])-1):
    dropoff = funnel_results['users'][i] - funnel_results['users'][i+1]
    dropoff_rate = (dropoff / funnel_results['users'][i]) * 100
    stage_dropoffs.append((funnel_results['stage'][i+1], dropoff, dropoff_rate))
    print(f"Drop-off at {funnel_results['stage'][i]} → {funnel_results['stage'][i+1]}: {dropoff:,} users ({dropoff_rate:.1f}%)")


In [ ]:
def conduct_ab_test_analysis(df):
    """
    Run A/B test analysis - morning vs evening notifications
    This got pretty long but covers all the main metrics we need
    """
    
    print("=== A/B TEST: MORNING vs EVENING NOTIFICATIONS ===")
    
    # Split the data - straightforward approach
    morning_users = df[df['notification_time'] == 'morning']
    evening_users = df[df['notification_time'] == 'evening']
    
    print(f"\\nSample sizes:")
    print(f"Morning: {len(morning_users):,} users")
    print(f"Evening: {len(evening_users):,} users") 
    print(f"Total: {len(df):,} users")
    
    # I'll store results in a dict
    results = {}
    
    # ===== Daily Sessions Analysis =====
    print(f"\\n1. DAILY SESSIONS")
    
    # Get the session data
    morning_sessions = morning_users['daily_sessions'].values
    evening_sessions = evening_users['daily_sessions'].values
    
    # Basic stats
    morning_avg = np.mean(morning_sessions)
    evening_avg = np.mean(evening_sessions)
    morning_sd = np.std(morning_sessions)
    evening_sd = np.std(evening_sessions)
    
    print(f"   Morning: {morning_avg:.2f} ± {morning_sd:.2f} sessions/day")
    print(f"   Evening: {evening_avg:.2f} ± {evening_sd:.2f} sessions/day")
    print(f"   Difference: {evening_avg - morning_avg:.2f} sessions/day")
    
    # T-test time
    t_stat, p_val = stats.ttest_ind(evening_sessions, morning_sessions)
    
    # Effect size - Cohen's d (this formula is a bit messy but works)
    n1, n2 = len(morning_sessions), len(evening_sessions)
    pooled_std = np.sqrt(((n1 - 1) * morning_sd**2 + (n2 - 1) * evening_sd**2) / (n1 + n2 - 2))
    cohens_d = (evening_avg - morning_avg) / pooled_std
    
    print(f"   t-stat: {t_stat:.3f}")
    print(f"   p-value: {p_val:.6f}")
    print(f"   Cohen's d: {cohens_d:.3f}")
    
    # 95% confidence interval 
    se_diff = pooled_std * np.sqrt(1/n1 + 1/n2)
    diff = evening_avg - morning_avg
    ci_low = diff - 1.96 * se_diff
    ci_high = diff + 1.96 * se_diff
    print(f"   95% CI: [{ci_low:.3f}, {ci_high:.3f}]")
    
    is_sig = p_val < 0.05
    print(f"   Result: {'SIGNIFICANT' if is_sig else 'NOT SIGNIFICANT'}")
    
    results['sessions'] = {
        'morning_mean': morning_avg,
        'evening_mean': evening_avg,
        'difference': diff,
        'p_value': p_val,
        'cohens_d': cohens_d,
        'significant': is_sig
    }
    
    # ===== Lesson Completion =====
    print(f"\\n2. LESSON COMPLETION")
    
    # Grab lesson data for both groups
    morning_lessons = morning_users['lesson_completed'].values
    evening_lessons = evening_users['lesson_completed'].values
    
    # Calculate means and stds
    morning_lessons_avg = np.mean(morning_lessons)
    evening_lessons_avg = np.mean(evening_lessons)
    morning_lessons_sd = np.std(morning_lessons)
    evening_lessons_sd = np.std(evening_lessons)
    
    print(f"   Morning: {morning_lessons_avg:.2f} ± {morning_lessons_sd:.2f} lessons")
    print(f"   Evening: {evening_lessons_avg:.2f} ± {evening_lessons_sd:.2f} lessons")
    
    # Difference
    lesson_diff = evening_lessons_avg - morning_lessons_avg
    print(f"   Difference: {lesson_diff:.2f} lessons")
    
    # Run t-test
    t_stat2, p_val2 = stats.ttest_ind(evening_lessons, morning_lessons)
    
    # Cohen's d again - using same approach as above
    n1_lessons = len(morning_lessons)
    n2_lessons = len(evening_lessons)
    pooled_std2 = np.sqrt(((n1_lessons - 1) * morning_lessons_sd**2 + 
                          (n2_lessons - 1) * evening_lessons_sd**2) / 
                          (n1_lessons + n2_lessons - 2))
    cohens_d2 = lesson_diff / pooled_std2
    
    print(f"   t-stat: {t_stat2:.3f}")
    print(f"   p-value: {p_val2:.6f}")
    print(f"   Cohen's d: {cohens_d2:.3f}")
    
    # Significance check
    sig2 = p_val2 < 0.05
    print(f"   Result: {'SIGNIFICANT' if sig2 else 'NOT SIGNIFICANT'}")
    
    results['lessons'] = {
        'morning_mean': morning_lessons_avg,
        'evening_mean': evening_lessons_avg,
        'difference': lesson_diff,
        'p_value': p_val2,
        'cohens_d': cohens_d2,
        'significant': sig2
    }
    
    # ===== Streak Milestones =====
    print(f"\\n3. STREAK MILESTONE ACHIEVEMENT")
    
    # Make a crosstab to see the breakdown
    streak_table = pd.crosstab(df['notification_time'], df['streak_milestone'])
    print(f"   Contingency Table:")
    print(streak_table)
    
    # Get the rates for each group
    morning_streak_pct = morning_users['streak_milestone'].mean()
    evening_streak_pct = evening_users['streak_milestone'].mean()
    
    print(f"\\n   Morning streak rate: {morning_streak_pct:.1%}")
    print(f"   Evening streak rate: {evening_streak_pct:.1%}")
    print(f"   Difference: {evening_streak_pct - morning_streak_pct:.1%}")
    
    # Chi-square test - this is for categorical data
    chi2_stat, chi2_p, dof, expected_vals = stats.chi2_contingency(streak_table)
    
    print(f"   Chi-square: {chi2_stat:.3f}")
    print(f"   p-value: {chi2_p:.6f}")
    print(f"   df: {dof}")
    
    is_sig_streak = chi2_p < 0.05
    print(f"   Result: {'SIGNIFICANT' if is_sig_streak else 'NOT SIGNIFICANT'}")
    
    results['streaks'] = {
        'morning_rate': morning_streak_pct,
        'evening_rate': evening_streak_pct,
        'difference': evening_streak_pct - morning_streak_pct,
        'p_value': chi2_p,
        'chi2_stat': chi2_stat,
        'significant': is_sig_streak
    }
    
    # ===== Churn Analysis =====
    print(f"\\n4. CHURN RATES")
    
    # Simple approach - just get the rates
    morning_churn = morning_users['churn_status'].mean()
    evening_churn = evening_users['churn_status'].mean()
    
    print(f"   Morning churn: {morning_churn:.1%}")
    print(f"   Evening churn: {evening_churn:.1%}")
    print(f"   Difference: {evening_churn - morning_churn:.1%}")
    
    # Another chi-square test
    churn_crosstab = pd.crosstab(df['notification_time'], df['churn_status'])
    chi2_churn, p_churn, dof_churn, exp_churn = stats.chi2_contingency(churn_crosstab)
    
    print(f"   Chi-square: {chi2_churn:.3f}")
    print(f"   p-value: {p_churn:.6f}")
    
    churn_sig = p_churn < 0.05
    print(f"   Result: {'SIGNIFICANT' if churn_sig else 'NOT SIGNIFICANT'}")
    
    results['churn'] = {
        'morning_rate': morning_churn,
        'evening_rate': evening_churn,
        'difference': evening_churn - morning_churn,
        'p_value': p_churn,
        'chi2_stat': chi2_churn,
        'significant': churn_sig
    }
    
    return results, morning_users, evening_users

# Run the A/B test analysis
ab_results, morning_users, evening_users = conduct_ab_test_analysis(df)
# Note: using df instead of df_processed since they should be the same at this point


In [ ]:
# Let me double-check these results make sense
print("Quick sanity checks:")
print(f"Total users: {len(df):,}")
print(f"Morning/Evening split: {len(morning_users)}/{len(evening_users)}")
print(f"Overall churn rate: {df['churn_status'].mean():.1%}")

# Check if the differences are practically meaningful
sessions_improvement = (ab_results['sessions']['evening_mean'] / ab_results['sessions']['morning_mean'] - 1) * 100
lessons_improvement = (ab_results['lessons']['evening_mean'] / ab_results['lessons']['morning_mean'] - 1) * 100

print(f"\\nPercentage improvements:")
print(f"Sessions: +{sessions_improvement:.1f}%")
print(f"Lessons: +{lessons_improvement:.1f}%")

# These look good - evening notifications are clearly better


In [ ]:
# Let me create some visualizations for the A/B test results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Daily Sessions - boxplot works well here
sessions_data = [morning_users['daily_sessions'], evening_users['daily_sessions']]
box1 = ax1.boxplot(sessions_data, labels=['Morning', 'Evening'], patch_artist=True)
box1['boxes'][0].set_facecolor('lightblue')  # morning = blue
box1['boxes'][1].set_facecolor('lightcoral') # evening = coral
ax1.set_title('Daily Sessions: Morning vs Evening\\n' + 
              f"p = {ab_results['sessions']['p_value']:.4f}", 
              fontweight='bold', fontsize=12)
ax1.set_ylabel('Sessions per Day')
ax1.grid(True, alpha=0.3)

# Add mean dots
ax1.scatter([1, 2], 
           [ab_results['sessions']['morning_mean'], 
            ab_results['sessions']['evening_mean']], 
           color='red', s=80, marker='D', label='Mean', zorder=5)
ax1.legend()

# 2. Lessons - similar approach
lessons_data = [morning_users['lesson_completed'], evening_users['lesson_completed']]
box2 = ax2.boxplot(lessons_data, labels=['Morning', 'Evening'], patch_artist=True)
box2['boxes'][0].set_facecolor('lightblue')
box2['boxes'][1].set_facecolor('lightcoral')
ax2.set_title('Lesson Completion\\n' + 
              f"p = {ab_results['lessons']['p_value']:.4f}", 
              fontweight='bold', fontsize=12)
ax2.set_ylabel('Lessons Completed')
ax2.grid(True, alpha=0.3)

# Mean markers again
ax2.scatter([1, 2], 
           [ab_results['lessons']['morning_mean'], 
            ab_results['lessons']['evening_mean']], 
           color='red', s=80, marker='D', label='Mean', zorder=5)
ax2.legend()

# 3. Streak rates - bar chart is better for percentages
streak_pcts = [ab_results['streaks']['morning_rate']*100, 
               ab_results['streaks']['evening_rate']*100]
bars1 = ax3.bar(['Morning', 'Evening'], streak_pcts, 
                color=['lightblue', 'lightcoral'], alpha=0.8, edgecolor='black')
ax3.set_title('Streak Achievement Rate\\n' + 
              f"p = {ab_results['streaks']['p_value']:.4f}", 
              fontweight='bold', fontsize=12)
ax3.set_ylabel('Achievement Rate (%)')
ax3.set_ylim(0, max(streak_pcts) * 1.15)

# Add percentage labels on bars
for bar, pct in zip(bars1, streak_pcts):
    ax3.text(bar.get_x() + bar.get_width()/2., pct + max(streak_pcts)*0.01,
             f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

# 4. Churn rates 
churn_pcts = [ab_results['churn']['morning_rate']*100, 
              ab_results['churn']['evening_rate']*100]
bars2 = ax4.bar(['Morning', 'Evening'], churn_pcts, 
                color=['lightblue', 'lightcoral'], alpha=0.8, edgecolor='black')
ax4.set_title('Churn Rate\\n' + 
              f"p = {ab_results['churn']['p_value']:.4f}", 
              fontweight='bold', fontsize=12)
ax4.set_ylabel('Churn Rate (%)')
ax4.set_ylim(0, max(churn_pcts) * 1.15)

# Labels on bars
for bar, pct in zip(bars2, churn_pcts):
    ax4.text(bar.get_x() + bar.get_width()/2., pct + max(churn_pcts)*0.01,
             f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Quick summary of what we found
print("\\n=== WHAT DID WE LEARN? ===")
print("Statistical significance (p < 0.05):")
print(f"• Daily Sessions: {'✓ YES' if ab_results['sessions']['significant'] else '✗ NO'}")
print(f"• Lessons: {'✓ YES' if ab_results['lessons']['significant'] else '✗ NO'}")  
print(f"• Streaks: {'✓ YES' if ab_results['streaks']['significant'] else '✗ NO'}")
print(f"• Churn: {'✓ YES' if ab_results['churn']['significant'] else '✗ NO'}")

print("\\nEffect sizes (Cohen's d for continuous vars):")
sessions_d = ab_results['sessions']['cohens_d']
lessons_d = ab_results['lessons']['cohens_d']

# Helper function to categorize effect size
def effect_size_label(d):
    if abs(d) > 0.8: return 'Large'
    elif abs(d) > 0.5: return 'Medium'
    elif abs(d) > 0.2: return 'Small'
    else: return 'Negligible'

print(f"• Sessions: {sessions_d:.3f} ({effect_size_label(sessions_d)})")
print(f"• Lessons: {lessons_d:.3f} ({effect_size_label(lessons_d)})")

print("\\nBottom line - what does this mean for business?")
sessions_diff = ab_results['sessions']['difference']
lessons_diff = ab_results['lessons']['difference'] 
streak_diff = ab_results['streaks']['difference'] * 100
churn_diff = ab_results['churn']['difference'] * 100

print(f"• Evening users have {sessions_diff:+.2f} more daily sessions")
print(f"• Evening users complete {lessons_diff:+.2f} more lessons")
print(f"• Streak achievement {'up' if streak_diff > 0 else 'down'} by {abs(streak_diff):.1f} percentage points")
print(f"• Churn rate {'increases' if churn_diff > 0 else 'decreases'} by {abs(churn_diff):.1f} percentage points")

print("\\n💡 Recommendation: Switch everyone to evening notifications!")


In [ ]:
def build_churn_prediction_model(df):
    """
    Build and evaluate a logistic regression model for churn prediction
    
    Parameters:
    df (pd.DataFrame): Processed user data
    
    Returns:
    tuple: (model, test_results, feature_importance)
    """
    
    print("=== CHURN PREDICTION MODEL ===")
    
    # Prepare features for modeling
    feature_columns = [
        'lesson_completed',
        'daily_sessions', 
        'days_since_signup',
        'engagement_score'
    ]
    
    # Add notification_time as binary feature
    df_model = df.copy()
    df_model['notification_evening'] = (df_model['notification_time'] == 'evening').astype(int)
    df_model['streak_milestone_int'] = df_model['streak_milestone'].astype(int)
    
    feature_columns.extend(['notification_evening', 'streak_milestone_int'])
    
    # Prepare feature matrix and target
    X = df_model[feature_columns]
    y = df_model['churn_status'].astype(int)
    
    print(f"Features used: {feature_columns}")
    print(f"Dataset shape: {X.shape}")
    print(f"Churn rate: {y.mean():.1%}")
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\\nTrain set: {X_train.shape[0]:,} samples")
    print(f"Test set: {X_test.shape[0]:,} samples")
    print(f"Train churn rate: {y_train.mean():.1%}")
    print(f"Test churn rate: {y_test.mean():.1%}")
    
    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train logistic regression model
    print("\\n=== MODEL TRAINING ===")
    lr_model = LogisticRegression(random_state=42, max_iter=1000)
    lr_model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = lr_model.predict(X_test_scaled)
    y_pred_proba = lr_model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    print(f"Model Performance:")
    print(f"  Accuracy:  {accuracy:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  F1-Score:  {f1:.3f}")
    print(f"  AUC-ROC:   {auc_score:.3f}")
    
    # Feature importance (coefficients)
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'coefficient': lr_model.coef_[0],
        'abs_coefficient': np.abs(lr_model.coef_[0])
    }).sort_values('abs_coefficient', ascending=False)
    
    print(f"\\n=== FEATURE IMPORTANCE ===")
    print("Features ranked by importance (absolute coefficient):")
    for idx, row in feature_importance.iterrows():
        direction = "↑ increases" if row['coefficient'] > 0 else "↓ decreases"
        print(f"  {row['feature']:<20}: {row['coefficient']:>7.3f} ({direction} churn risk)")
    
    # Create detailed results dictionary
    results = {
        'model': lr_model,
        'scaler': scaler,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_score,
        'feature_importance': feature_importance,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'feature_columns': feature_columns
    }
    
    return results

# Build and evaluate churn prediction model
churn_model_results = build_churn_prediction_model(df_processed)

# Confusion Matrix and Performance Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(churn_model_results['y_test'], churn_model_results['y_pred'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Retained', 'Churned'], 
            yticklabels=['Retained', 'Churned'])
ax1.set_title('Confusion Matrix', fontweight='bold')
ax1.set_xlabel('Predicted')
ax1.set_ylabel('Actual')

# 2. ROC Curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(churn_model_results['y_test'], churn_model_results['y_pred_proba'])
ax2.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {churn_model_results["auc_roc"]:.3f})')
ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random Classifier')
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curve', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Feature Importance
importance_df = churn_model_results['feature_importance']
bars = ax3.barh(range(len(importance_df)), importance_df['coefficient'], 
                color=['red' if x > 0 else 'blue' for x in importance_df['coefficient']])
ax3.set_yticks(range(len(importance_df)))
ax3.set_yticklabels(importance_df['feature'])
ax3.set_xlabel('Coefficient Value')
ax3.set_title('Feature Importance\\n(Positive = Increases Churn Risk)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(importance_df.iterrows()):
    ax3.text(row['coefficient'] + 0.01 if row['coefficient'] > 0 else row['coefficient'] - 0.01,
             i, f'{row["coefficient"]:.3f}', 
             va='center', ha='left' if row['coefficient'] > 0 else 'right')

# 4. Predicted Probability Distribution
ax4.hist(churn_model_results['y_pred_proba'][churn_model_results['y_test'] == 0], 
         bins=30, alpha=0.7, label='Retained Users', color='blue', density=True)
ax4.hist(churn_model_results['y_pred_proba'][churn_model_results['y_test'] == 1], 
         bins=30, alpha=0.7, label='Churned Users', color='red', density=True)
ax4.set_xlabel('Predicted Churn Probability')
ax4.set_ylabel('Density')
ax4.set_title('Predicted Probability Distribution', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n=== MODEL INTERPRETATION ===")
print("Key Insights:")
top_features = churn_model_results['feature_importance'].head(3)
for idx, row in top_features.iterrows():
    impact = "increases" if row['coefficient'] > 0 else "decreases"
    print(f"• {row['feature']}: {impact} churn risk significantly (coef: {row['coefficient']:.3f})")

print(f"\\nModel Reliability:")
print(f"• AUC-ROC of {churn_model_results['auc_roc']:.3f} indicates {'excellent' if churn_model_results['auc_roc'] > 0.9 else 'good' if churn_model_results['auc_roc'] > 0.8 else 'fair' if churn_model_results['auc_roc'] > 0.7 else 'poor'} discriminative ability")
print(f"• Precision of {churn_model_results['precision']:.1%} means {churn_model_results['precision']:.1%} of predicted churners actually churn")
print(f"• Recall of {churn_model_results['recall']:.1%} means we identify {churn_model_results['recall']:.1%} of all actual churners")


In [ ]:
def simulate_realtime_dashboard(df, n_recent_events=1000):
    """
    Simulate real-time dashboard with recent user events and hourly trends
    
    Parameters:
    df (pd.DataFrame): Full processed dataset
    n_recent_events (int): Number of recent events to simulate
    
    Returns:
    dict: Real-time dashboard data and visualizations
    """
    
    print("=== REAL-TIME DASHBOARD SIMULATION ===")
    
    # Simulate "recent" events (last 1000 users by signup date)
    df_recent = df.nlargest(n_recent_events, 'signup_date').copy()
    
    print(f"Simulating dashboard with {len(df_recent):,} recent events")
    print(f"Date range: {df_recent['signup_date'].min().strftime('%Y-%m-%d')} to {df_recent['signup_date'].max().strftime('%Y-%m-%d')}")
    
    # Generate hourly timestamps for simulation
    end_time = datetime.now()
    start_time = end_time - timedelta(days=7)  # Last 7 days
    hourly_range = pd.date_range(start=start_time, end=end_time, freq='H')
    
    # Simulate hourly metrics
    np.random.seed(42)  # For reproducible simulation
    
    # Daily Active Users (DAU) - simulate hourly variation
    base_dau = len(df_recent) // 7  # Average daily active users
    dau_hourly = []
    
    for hour in hourly_range:
        # Add realistic hourly patterns
        hour_of_day = hour.hour
        day_of_week = hour.weekday()
        
        # Peak hours: 8-10 AM and 7-9 PM
        if hour_of_day in [8, 9, 19, 20]:
            multiplier = 1.3
        elif hour_of_day in [10, 11, 18, 21]:
            multiplier = 1.1
        elif hour_of_day in range(1, 6):  # Low activity hours
            multiplier = 0.3
        else:
            multiplier = 1.0
            
        # Weekend effect
        if day_of_week in [5, 6]:  # Saturday, Sunday
            multiplier *= 0.8
            
        # Add random variation
        noise = np.random.normal(1, 0.1)
        simulated_dau = int(base_dau * multiplier * noise)
        dau_hourly.append(max(1, simulated_dau))
    
    # Lesson completion rate (lessons per hour)
    lesson_completion_hourly = []
    for i, dau in enumerate(dau_hourly):
        # Assume 0.3-0.8 lessons per active user per hour on average
        avg_lessons_per_user = np.random.uniform(0.3, 0.8)
        hourly_lessons = int(dau * avg_lessons_per_user)
        lesson_completion_hourly.append(hourly_lessons)
    
    # Create dashboard data
    dashboard_data = pd.DataFrame({
        'timestamp': hourly_range,
        'dau': dau_hourly,
        'lesson_completions': lesson_completion_hourly
    })
    
    # Calculate rolling averages
    dashboard_data['dau_rolling_24h'] = dashboard_data['dau'].rolling(24, min_periods=1).mean()
    dashboard_data['lessons_rolling_24h'] = dashboard_data['lesson_completions'].rolling(24, min_periods=1).mean()
    
    # Engagement alerts (simulate high-risk users)
    high_risk_users = df_recent[df_recent['churn_risk_score'] > 0.7]
    medium_risk_users = df_recent[(df_recent['churn_risk_score'] > 0.5) & (df_recent['churn_risk_score'] <= 0.7)]
    
    print(f"\\n=== REAL-TIME ALERTS ===")
    print(f"High Risk Users: {len(high_risk_users):,} ({len(high_risk_users)/len(df_recent)*100:.1f}%)")
    print(f"Medium Risk Users: {len(medium_risk_users):,} ({len(medium_risk_users)/len(df_recent)*100:.1f}%)")
    
    # Current metrics
    current_dau = dau_hourly[-1]
    current_lessons = lesson_completion_hourly[-1]
    avg_dau_24h = np.mean(dau_hourly[-24:])
    avg_lessons_24h = np.mean(lesson_completion_hourly[-24:])
    
    print(f"\\n=== CURRENT METRICS ===")
    print(f"Current Hour DAU: {current_dau:,}")
    print(f"24h Average DAU: {avg_dau_24h:.0f}")
    print(f"Current Hour Lessons: {current_lessons:,}")
    print(f"24h Average Lessons: {avg_lessons_24h:.0f}")
    
    return dashboard_data, high_risk_users, medium_risk_users

# Generate real-time dashboard simulation
dashboard_data, high_risk_users, medium_risk_users = simulate_realtime_dashboard(df_processed, n_recent_events=1000)

# Create real-time dashboard visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))

# 1. Daily Active Users Trend
ax1.plot(dashboard_data['timestamp'], dashboard_data['dau'], 
         color='blue', alpha=0.6, linewidth=1, label='Hourly DAU')
ax1.plot(dashboard_data['timestamp'], dashboard_data['dau_rolling_24h'], 
         color='red', linewidth=2, label='24h Rolling Average')
ax1.set_title('Daily Active Users - Real-time Trend', fontweight='bold', fontsize=12)
ax1.set_ylabel('Active Users')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Add current value annotation
current_dau = dashboard_data['dau'].iloc[-1]
ax1.annotate(f'Current: {current_dau}', 
             xy=(dashboard_data['timestamp'].iloc[-1], current_dau),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# 2. Lesson Completion Trend
ax2.plot(dashboard_data['timestamp'], dashboard_data['lesson_completions'], 
         color='green', alpha=0.6, linewidth=1, label='Hourly Completions')
ax2.plot(dashboard_data['timestamp'], dashboard_data['lessons_rolling_24h'], 
         color='orange', linewidth=2, label='24h Rolling Average')
ax2.set_title('Lesson Completions - Real-time Trend', fontweight='bold', fontsize=12)
ax2.set_ylabel('Lessons Completed')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# 3. User Risk Distribution
risk_categories = ['Low Risk', 'Medium Risk', 'High Risk']
risk_counts = [
    len(df_processed) - len(medium_risk_users) - len(high_risk_users),
    len(medium_risk_users),
    len(high_risk_users)
]
colors = ['green', 'orange', 'red']

wedges, texts, autotexts = ax3.pie(risk_counts, labels=risk_categories, colors=colors, 
                                   autopct='%1.1f%%', startangle=90)
ax3.set_title('User Churn Risk Distribution\\n(Recent 1000 Users)', fontweight='bold', fontsize=12)

# 4. A/B Test Performance (Real-time)
recent_morning = df_processed[df_processed['notification_time'] == 'morning'].tail(500)
recent_evening = df_processed[df_processed['notification_time'] == 'evening'].tail(500)

ab_metrics = {
    'Morning': [
        recent_morning['daily_sessions'].mean(),
        recent_morning['lesson_completed'].mean(),
        recent_morning['streak_milestone'].mean() * 100
    ],
    'Evening': [
        recent_evening['daily_sessions'].mean(),
        recent_evening['lesson_completed'].mean(), 
        recent_evening['streak_milestone'].mean() * 100
    ]
}

x = np.arange(len(['Daily Sessions', 'Lessons Completed', 'Streak Rate (%)']))
width = 0.35

morning_bars = ax4.bar(x - width/2, ab_metrics['Morning'], width, 
                      label='Morning', color='lightblue', alpha=0.7)
evening_bars = ax4.bar(x + width/2, ab_metrics['Evening'], width,
                      label='Evening', color='lightcoral', alpha=0.7)

ax4.set_xlabel('Metrics')
ax4.set_ylabel('Values')
ax4.set_title('A/B Test Performance - Recent Users', fontweight='bold', fontsize=12)
ax4.set_xticks(x)
ax4.set_xticklabels(['Daily Sessions', 'Lessons Completed', 'Streak Rate (%)'])
ax4.legend()
ax4.grid(True, alpha=0.3)

# Add value labels on bars
for bars in [morning_bars, evening_bars]:
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Print real-time alerts and recommendations
print("\\n=== REAL-TIME DASHBOARD SUMMARY ===")
print(f"Dashboard Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\\nKEY PERFORMANCE INDICATORS:")
print(f"• Current DAU: {dashboard_data['dau'].iloc[-1]:,}")
print(f"• 24h Average DAU: {dashboard_data['dau_rolling_24h'].iloc[-1]:.0f}")
print(f"• Hourly Lesson Completions: {dashboard_data['lesson_completions'].iloc[-1]:,}")
print(f"• Lessons per Active User: {dashboard_data['lesson_completions'].iloc[-1] / dashboard_data['dau'].iloc[-1]:.2f}")

print("\\nRISK ALERTS:")
if len(high_risk_users) > len(df_processed) * 0.1:
    print(f"⚠️  HIGH ALERT: {len(high_risk_users):,} users at high churn risk ({len(high_risk_users)/len(df_processed)*100:.1f}%)")
else:
    print(f"✅ Churn risk within acceptable range: {len(high_risk_users):,} high-risk users ({len(high_risk_users)/len(df_processed)*100:.1f}%)")

print("\\nACTIONABLE INSIGHTS:")
print("• Monitor DAU trends for unusual drops during peak hours")
print("• Focus retention efforts on high-risk user segment")
print("• Continue A/B testing to optimize notification timing")
print("• Consider personalized interventions for medium-risk users")


In [ ]:
def prepare_bi_export_dataset(df):
    """
    Prepare cleaned, enriched dataset optimized for BI tools like Tableau/Power BI
    
    Parameters:
    df (pd.DataFrame): Processed dataset with all features
    
    Returns:
    pd.DataFrame: BI-ready dataset with clean column names and optimized structure
    """
    
    print("=== PREPARING BI EXPORT DATASET ===")
    
    # Create copy for export preparation
    df_export = df.copy()
    
    # Rename columns for BI tool compatibility (no spaces, descriptive names)
    column_mapping = {
        'user_id': 'User_ID',
        'signup_date': 'Signup_Date',
        'lesson_completed': 'Lessons_Completed',
        'streak_milestone': 'Streak_Milestone_Achieved',
        'daily_sessions': 'Daily_Sessions_Avg',
        'notification_time': 'Notification_Group',
        'churn_status': 'Churned_30_Day',
        'days_since_signup': 'User_Tenure_Days',
        'cumulative_lessons': 'Total_Lessons_Completed',
        'engagement_score': 'Engagement_Score',
        'user_segment': 'User_Engagement_Segment',
        'churn_risk_score': 'Churn_Risk_Score',
        'signup_month': 'Signup_Month',
        'signup_week': 'Signup_Week'
    }
    
    df_export = df_export.rename(columns=column_mapping)
    
    # Add calculated fields commonly used in BI analysis
    
    # 1. Engagement Level (categorical)
    def categorize_engagement(score):
        if score >= 15:
            return 'High'
        elif score >= 8:
            return 'Medium' 
        elif score >= 3:
            return 'Low'
        else:
            return 'At Risk'
    
    df_export['Engagement_Level'] = df_export['Engagement_Score'].apply(categorize_engagement)
    
    # 2. Churn Risk Category
    def categorize_churn_risk(score):
        if score >= 0.7:
            return 'High Risk'
        elif score >= 0.4:
            return 'Medium Risk'
        else:
            return 'Low Risk'
    
    df_export['Churn_Risk_Category'] = df_export['Churn_Risk_Score'].apply(categorize_churn_risk)
    
    # 3. User Lifecycle Stage based on tenure
    def categorize_lifecycle(days):
        if days <= 7:
            return 'New User'
        elif days <= 30:
            return 'Growing User'
        elif days <= 90:
            return 'Established User'
        else:
            return 'Long-term User'
    
    df_export['User_Lifecycle_Stage'] = df_export['User_Tenure_Days'].apply(categorize_lifecycle)
    
    # 4. Performance vs Benchmark
    avg_lessons = df_export['Lessons_Completed'].mean()
    avg_sessions = df_export['Daily_Sessions_Avg'].mean()
    
    df_export['Lessons_vs_Average'] = (df_export['Lessons_Completed'] / avg_lessons).round(2)
    df_export['Sessions_vs_Average'] = (df_export['Daily_Sessions_Avg'] / avg_sessions).round(2)
    
    # 5. High Performer Flag
    df_export['High_Performer'] = (
        (df_export['Lessons_vs_Average'] > 1.5) & 
        (df_export['Sessions_vs_Average'] > 1.2)
    )
    
    # 6. Date components for time-based analysis
    df_export['Signup_Year'] = df_export['Signup_Date'].dt.year
    df_export['Signup_Month_Name'] = df_export['Signup_Date'].dt.strftime('%B')
    df_export['Signup_Quarter'] = df_export['Signup_Date'].dt.quarter
    df_export['Signup_Day_of_Week'] = df_export['Signup_Date'].dt.day_name()
    
    # 7. Binary flags for easier filtering
    df_export['Notification_Evening_Flag'] = (df_export['Notification_Group'] == 'evening').astype(int)
    df_export['Streak_Achieved_Flag'] = df_export['Streak_Milestone_Achieved'].astype(int)
    df_export['Churned_Flag'] = df_export['Churned_30_Day'].astype(int)
    
    # Optimize data types for export
    # Convert categorical columns to string for better BI tool compatibility
    categorical_columns = [
        'Notification_Group', 'User_Engagement_Segment', 'Engagement_Level',
        'Churn_Risk_Category', 'User_Lifecycle_Stage', 'Signup_Month',
        'Signup_Month_Name', 'Signup_Day_of_Week'
    ]
    
    for col in categorical_columns:
        if col in df_export.columns:
            df_export[col] = df_export[col].astype(str)
    
    # Ensure boolean columns are properly formatted
    boolean_columns = [
        'Streak_Milestone_Achieved', 'Churned_30_Day', 'High_Performer'
    ]
    
    for col in boolean_columns:
        if col in df_export.columns:
            df_export[col] = df_export[col].astype(bool)
    
    # Round numeric columns to appropriate precision
    numeric_columns = [
        'Engagement_Score', 'Churn_Risk_Score', 'Daily_Sessions_Avg',
        'Lessons_vs_Average', 'Sessions_vs_Average'
    ]
    
    for col in numeric_columns:
        if col in df_export.columns:
            df_export[col] = df_export[col].round(3)
    
    # Sort by signup date for logical ordering
    df_export = df_export.sort_values('Signup_Date')
    
    print(f"Export dataset prepared with {df_export.shape[0]:,} rows and {df_export.shape[1]} columns")
    print(f"Date range: {df_export['Signup_Date'].min().strftime('%Y-%m-%d')} to {df_export['Signup_Date'].max().strftime('%Y-%m-%d')}")
    
    return df_export

# Prepare BI export dataset
df_bi_export = prepare_bi_export_dataset(df_processed)

# Create data dictionary for BI users
data_dictionary = {
    'Column_Name': [],
    'Description': [],
    'Data_Type': [],
    'Sample_Values': []
}

column_descriptions = {
    'User_ID': 'Unique identifier for each user',
    'Signup_Date': 'Date when user registered for the app',
    'Lessons_Completed': 'Total number of lessons completed by user',
    'Streak_Milestone_Achieved': 'Whether user achieved streak milestones (True/False)',
    'Daily_Sessions_Avg': 'Average number of daily app sessions',
    'Notification_Group': 'A/B test group assignment (morning/evening)',
    'Churned_30_Day': 'Whether user churned within 30 days (True/False)',
    'User_Tenure_Days': 'Number of days since user signup',
    'Engagement_Score': 'Composite engagement metric (0-30 scale)',
    'User_Engagement_Segment': 'User categorization based on engagement',
    'Churn_Risk_Score': 'Predicted churn probability (0-1 scale)',
    'Engagement_Level': 'Simplified engagement categorization',
    'Churn_Risk_Category': 'Risk level categorization',
    'User_Lifecycle_Stage': 'User maturity based on tenure',
    'Lessons_vs_Average': 'User performance vs dataset average',
    'Sessions_vs_Average': 'Session frequency vs dataset average',
    'High_Performer': 'Flag for users exceeding performance thresholds',
    'Signup_Year': 'Year of user signup',
    'Signup_Month_Name': 'Month name of signup',
    'Signup_Quarter': 'Quarter of signup (1-4)',
    'Signup_Day_of_Week': 'Day of week for signup'
}

for col in df_bi_export.columns:
    data_dictionary['Column_Name'].append(col)
    data_dictionary['Description'].append(column_descriptions.get(col, 'Derived field for analysis'))
    data_dictionary['Data_Type'].append(str(df_bi_export[col].dtype))
    
    # Sample values
    if df_bi_export[col].dtype == 'object' or df_bi_export[col].dtype.name == 'category':
        sample_vals = df_bi_export[col].unique()[:3]
        data_dictionary['Sample_Values'].append(', '.join([str(v) for v in sample_vals]))
    else:
        sample_vals = df_bi_export[col].head(3).values
        data_dictionary['Sample_Values'].append(', '.join([str(v) for v in sample_vals]))

dd_df = pd.DataFrame(data_dictionary)

# Export to CSV
print("\\n=== EXPORTING DATA ===")
export_filename = f"gamified_learning_bi_export_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_bi_export.to_csv(export_filename, index=False)

# Export data dictionary
dict_filename = f"data_dictionary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
dd_df.to_csv(dict_filename, index=False)

print(f"✅ Main dataset exported to: {export_filename}")
print(f"✅ Data dictionary exported to: {dict_filename}")
print(f"   File size: {round(len(df_bi_export) * len(df_bi_export.columns) * 8 / 1024 / 1024, 2)} MB (estimated)")

# Show export preview
print("\\n=== EXPORT DATASET PREVIEW ===")
print("First 5 rows of exported dataset:")
display_cols = ['User_ID', 'Signup_Date', 'Lessons_Completed', 'Engagement_Level', 
                'Churn_Risk_Category', 'Notification_Group']
print(df_bi_export[display_cols].head())

print("\\n=== DATA DICTIONARY PREVIEW ===")
print("Column documentation:")
print(dd_df.head(10))


In [ ]:
# Final Analysis Summary with Quantified Business Impact
print("=" * 80)
print("GAMIFIED LEARNING APP: COMPREHENSIVE ANALYSIS SUMMARY")
print("=" * 80)

# Collect key metrics from our analysis
total_users = len(df_processed)
overall_churn_rate = df_processed['churn_status'].mean()
high_risk_users_count = len(df_processed[df_processed['churn_risk_score'] > 0.7])

# A/B test impact
evening_advantage_sessions = ab_results['daily_sessions']['difference']
evening_advantage_lessons = ab_results['lesson_completion']['difference']

# Funnel conversion rates
signup_to_streak_rate = (funnel_results['users'][-1] / funnel_results['users'][0]) * 100

# Model performance
model_auc = churn_model_results['auc_roc']
model_precision = churn_model_results['precision']

print(f"\\nDATASET OVERVIEW:")
print(f"• Total Users Analyzed: {total_users:,}")
print(f"• Analysis Period: 6 months")
print(f"• Data Quality: High (no missing values)")
print(f"• Overall Churn Rate: {overall_churn_rate:.1%}")

print(f"\\nCONVERSION FUNNEL PERFORMANCE:")
print(f"• Signup → Streak Achiever: {signup_to_streak_rate:.1f}%")
print(f"• Biggest Drop-off: First lesson → Active learner stage")
print(f"• Opportunity: ~{(funnel_results['users'][0] - funnel_results['users'][-1]):,} users lost through funnel")

print(f"\\nA/B TEST BUSINESS IMPACT:")
if ab_results['daily_sessions']['significant']:
    print(f"• Evening notifications: +{evening_advantage_sessions:.2f} daily sessions per user")
else:
    print(f"• Evening notifications: No significant difference in daily sessions")

if ab_results['lesson_completion']['significant']:
    print(f"• Evening notifications: +{evening_advantage_lessons:.2f} lessons per user")
else:
    print(f"• Evening notifications: No significant difference in lesson completion")

# Estimate revenue impact (hypothetical)
if evening_advantage_lessons > 0:
    potential_additional_lessons = evening_advantage_lessons * (total_users // 2)
    print(f"• Potential additional lessons per month: {potential_additional_lessons:,.0f}")

print(f"\\nCHURN PREDICTION MODEL:")
print(f"• Predictive Accuracy: {model_auc:.1%} AUC-ROC")
print(f"• Precision: {model_precision:.1%} (of predicted churners, {model_precision:.1%} actually churn)")
print(f"• High-Risk Users Identified: {high_risk_users_count:,} ({high_risk_users_count/total_users*100:.1f}%)")
print(f"• Model Reliability: {'Excellent' if model_auc > 0.9 else 'Good' if model_auc > 0.8 else 'Fair'}")

print(f"\\nREAL-TIME MONITORING:")
print(f"• Dashboard refresh capability: Hourly")
print(f"• Risk alert threshold: {(df_processed['churn_risk_score'] > 0.7).sum():,} users currently high-risk")
print(f"• Engagement tracking: Real-time DAU and lesson completion rates")

print("\\n" + "=" * 80)
print("STRATEGIC RECOMMENDATIONS")
print("=" * 80)

print("\\n1. IMMEDIATE ACTIONS (0-30 days):")
print("   ✓ Implement evening notification strategy for new users")
print("   ✓ Deploy churn prediction model for daily risk scoring")
print("   ✓ Create automated alerts for high-risk user segments")
print("   ✓ Set up real-time dashboard for operations team")

print("\\n2. SHORT-TERM INITIATIVES (1-3 months):")
print("   ✓ Develop targeted retention campaigns for medium-risk users")
print("   ✓ A/B test additional engagement strategies (push frequency, content)")
print("   ✓ Implement funnel optimization at key drop-off points")
print("   ✓ Create personalized learning paths based on engagement scores")

print("\\n3. LONG-TERM STRATEGY (3-12 months):")
print("   ✓ Build advanced ML models incorporating behavioral sequences")
print("   ✓ Develop predictive lifetime value models")
print("   ✓ Create dynamic user segmentation for personalized experiences")
print("   ✓ Implement closed-loop feedback system for continuous optimization")

print("\\n4. SUCCESS METRICS TO TRACK:")
print("   • Overall churn rate reduction target: -15% within 6 months")
print("   • Funnel conversion improvement: +20% signup to streak achiever")
print("   • Early prediction accuracy: Maintain >80% AUC-ROC")
print("   • Engagement score improvement: +10% average user engagement")

print("\\n5. RESOURCE REQUIREMENTS:")
print("   • Data Engineering: Real-time pipeline development")
print("   • Product Team: Feature implementation and A/B testing")
print("   • Marketing: Retention campaign development and execution")
print("   • Analytics: Model monitoring and performance tracking")

print("\\n" + "=" * 80)
print("BUSINESS VALUE ESTIMATION")
print("=" * 80)

# Hypothetical business value calculations
monthly_active_users = total_users * 0.6  # Assume 60% monthly active
avg_session_value = 0.05  # Hypothetical $0.05 per session
monthly_lesson_value = 0.25  # Hypothetical $0.25 per lesson

if evening_advantage_sessions > 0:
    additional_session_revenue = (evening_advantage_sessions * 30 * 
                                monthly_active_users * avg_session_value)
    print(f"\\nPOTENTIAL MONTHLY REVENUE IMPACT:")
    print(f"• Additional session revenue: ${additional_session_revenue:,.0f}")

if evening_advantage_lessons > 0:
    additional_lesson_revenue = (evening_advantage_lessons * 30 * 
                               monthly_active_users * monthly_lesson_value)
    print(f"• Additional lesson revenue: ${additional_lesson_revenue:,.0f}")

# Churn reduction value
current_monthly_churn = total_users * overall_churn_rate / 6  # 6 months of data
hypothetical_user_value = 15  # $15 LTV per retained user
potential_churn_reduction = current_monthly_churn * 0.15  # 15% reduction
churn_reduction_value = potential_churn_reduction * hypothetical_user_value

print(f"• Churn reduction value: ${churn_reduction_value:,.0f}/month")
print(f"• Total estimated monthly impact: ${(additional_session_revenue if evening_advantage_sessions > 0 else 0) + (additional_lesson_revenue if evening_advantage_lessons > 0 else 0) + churn_reduction_value:,.0f}")

print("\\n" + "=" * 80)
print("ANALYSIS COMPLETED - DATA EXPORTED FOR TABLEAU/POWER BI")
print("=" * 80)
print(f"\\nFiles generated:")
print(f"• Main dataset: {export_filename}")
print(f"• Data dictionary: {dict_filename}")
print(f"• Total records: {len(df_bi_export):,}")
print(f"• Ready for business intelligence visualization")
print("\\nAnalysis Date:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
